### Classificação de digitos escritos a mão para a base Mnist

In [1]:
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


#### Extração da base de dados e preprocessamento

In [2]:
(X_treinamento, y_treinamento), (X_teste, y_teste) = mnist.load_data()
previsores_treinamento = X_treinamento.reshape(X_treinamento.shape[0],
                                               28, 28, 1)
previsores_teste = X_teste.reshape(X_teste.shape[0], 28, 28, 1)
previsores_treinamento = previsores_treinamento.astype('float32')
previsores_teste = previsores_teste.astype('float32')
previsores_treinamento /= 255
previsores_teste /= 255
classe_treinamento = np_utils.to_categorical(y_treinamento, 10)
classe_teste = np_utils.to_categorical(y_teste, 10)

#### Estrutura da rede convolucional

In [3]:
classificador = Sequential()
classificador.add(Conv2D(32, (3,3), input_shape=(28, 28, 1), activation = 'relu'))
classificador.add(MaxPooling2D(pool_size = (2,2)))
classificador.add(Flatten())

classificador.add(Dense(units = 128, activation = 'relu'))
classificador.add(Dense(units = 10, activation = 'softmax'))
classificador.compile(loss = 'categorical_crossentropy',
                      optimizer = 'adam', metrics = ['accuracy'])

#### Gerando imagens em diferentes inclinações, rotações e zoom

In [4]:
gerador_treinamento = ImageDataGenerator(rotation_range = 7,
                                         horizontal_flip = True,
                                         shear_range = 0.2,
                                         height_shift_range = 0.07,
                                         zoom_range = 0.2)
gerador_teste = ImageDataGenerator()

In [5]:
base_treinamento = gerador_treinamento.flow(previsores_treinamento,
                                            classe_treinamento, batch_size = 128)
base_teste = gerador_teste.flow(previsores_teste, classe_teste, batch_size = 128)

#### Treinamento do modelo

In [6]:
classificador.fit_generator(base_treinamento, steps_per_epoch = 600000 / 128,
                            epochs = 5, validation_data = base_teste,
                            validation_steps = 10000 / 128)

Epoch 1/5
4688/4687 [==============================] - 468s 100ms/step - loss: 0.1696 - accuracy: 0.9472 - val_loss: 0.0816 - val_accuracy: 0.9826
Epoch 2/5
4688/4687 [==============================] - 495s 106ms/step - loss: 0.0638 - accuracy: 0.9799 - val_loss: 0.1292 - val_accuracy: 0.9835
Epoch 3/5
4688/4687 [==============================] - 492s 105ms/step - loss: 0.0473 - accuracy: 0.9850 - val_loss: 0.0593 - val_accuracy: 0.9882
Epoch 4/5
4688/4687 [==============================] - 480s 102ms/step - loss: 0.0390 - accuracy: 0.9875 - val_loss: 0.0973 - val_accuracy: 0.9875
Epoch 5/5
4688/4687 [==============================] - 492s 105ms/step - loss: 0.0334 - accuracy: 0.9892 - val_loss: 8.4177e-04 - val_accuracy: 0.9875


#### Predição para a base de teste e resultados

In [8]:
resultado = classificador.evaluate(previsores_teste, classe_teste)

10000/10000 [==============================] - 2s 165us/step


In [10]:
resultado

[0.03899515640666941, 0.987500011920929]